<a href="https://colab.research.google.com/github/arampacha/reformer_fastai/blob/master/nbs/exploration/_LM_timing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jan  6 16:07:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai einops datasets
    !pip install -qq git+git://github.com/arampacha/reformer_fastai.git
    !pip install -qq git+git://github.com/butchland/my_timesaver_utils.git

In [ ]:
from fastai.text.all import *

In [ ]:
from reformer_fastai.tokenizers import ByteTextTokenizer
from reformer_fastai.transformer import TransformerLM
from reformer_fastai.reformer import ReversibleLM
from reformer_fastai.metrics import bpc
from reformer_fastai.core import total_params
from my_timesaver_utils.all import MyProfileCallback

## Prepare data

In [ ]:
from datasets import load_dataset
dataset = load_dataset('tiny_shakespeare')

Using custom data configuration default
Reusing dataset tiny_shakespeare (/root/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/da11b9a200ca715af415094b34d5c956170f184cb3a58d7789680cf81bede955)


In [ ]:
train_ds = dataset['train']

In [ ]:
def splitlines(sample):
    return {'line':[*sample['text'][0].split('\n')]}

In [ ]:
train_ds = train_ds.map(splitlines, batched=True, remove_columns=['text'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/da11b9a200ca715af415094b34d5c956170f184cb3a58d7789680cf81bede955/cache-442ca6fc9dbbbfc8.arrow


In [ ]:
train_ds = train_ds.filter(lambda x: x['line'] != '')

Loading cached processed dataset at /root/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/da11b9a200ca715af415094b34d5c956170f184cb3a58d7789680cf81bede955/cache-29b7ecbdc2b09017.arrow


In [ ]:
df = train_ds.data.to_pandas()
# df.head()

In [ ]:
bte = ByteTextTokenizer(is_lm=True, add_bos=True, add_eos=True)
vocab_sz = bte.vocab_size

In [ ]:
@patch
def create_item(self:LMDataLoader, seq):
    if seq>=self.n: raise IndexError
    sl = self.last_len if seq//self.bs==self.n_batches-1 else self.seq_len
    st = (seq%self.bs)*self.bl + (seq//self.bs)*self.seq_len
    txt = self.chunks[st : st+sl+1]    
    return LMTensorText(txt[:-1]),txt[1:]

In [ ]:
cut = int(len(df)*0.8)
splits = range_of(df)[:cut], range_of(df[cut:])

In [ ]:
tfms = [attrgetter("line"), bte]
dsets = Datasets(df, [tfms, tfms], splits=splits, dl_type=LMDataLoader)

In [ ]:
bs,sl = 16,128
pad_seq2seq = partial(pad_input, pad_idx=bte.pad_token_id, pad_fields=[0,1])

dls = dsets.dataloaders(bs=bs, seq_len=sl, before_batch=pad_seq2seq)

In [ ]:
dls.show_batch(max_n=2)

,text,text_
0,"<bos>O comfortable friar! where is my lord?<eos><bos>pity: yet I'll tarry till my son come; he hallooed<eos><bos>Mortal, to cut it off; to cure it,","O comfortable friar! where is my lord?<eos><bos>pity: yet I'll tarry till my son come; he hallooed<eos><bos>Mortal, to cut it off; to cure it, e"
1,":<eos><bos>MONTAGUE:<eos><bos>My words are dull; O, quicken them with thine!<eos><bos>And harsh in sound to thine.<eos><bos>To-night she is mew'd up to her heav","<eos><bos>MONTAGUE:<eos><bos>My words are dull; O, quicken them with thine!<eos><bos>And harsh in sound to thine.<eos><bos>To-night she is mew'd up to her heavi"


## TransformerLM timing

In [ ]:
learn = Learner(dls, TransformerLM(vocab_sz, 512),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy, perplexity, bpc]).to_native_fp16()
total_parameters(learn.model)

In [ ]:
learn.fit_one_cycle(1, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,3.350112,3.345185,0.145055,28.365831,4.826082,03:52


In [ ]:
learn.to_my_profile()
learn.fit_one_cycle(2, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,3.358690,3.345467,0.145055,28.373833,4.826489,03:52
1,3.347866,3.343987,0.145055,28.331861,4.824353,03:52


In [ ]:
learn.my_profile.print_stats()

fit  called 1 times. max: 465.105 avg: 465.105
   epoch  called 2 times. max: 232.679 avg: 232.549
      train  called 2 times. max: 209.641 avg: 209.403
         train_batch  called 808 times. max: 0.557 avg: 0.413
            train_pred  called 808 times. max: 0.179 avg: 0.122
            train_loss  called 808 times. max: 0.003 avg: 0.001
            train_backward  called 808 times. max: 0.349 avg: 0.261
            train_step  called 808 times. max: 0.036 avg: 0.024
            train_zero_grad  called 808 times. max: 0.013 avg: 0.005
      valid  called 2 times. max: 23.243 avg: 23.136
         valid_batch  called 190 times. max: 0.139 avg: 0.135
            valid_pred  called 190 times. max: 0.122 avg: 0.118
            valid_loss  called 190 times. max: 0.002 avg: 0.001


In [ ]:
learn.my_profile.clear_stats()

In [ ]:
del learn
torch.cuda.empty_cache()

## ReversibleLM timing

In [ ]:
learn = Learner(dls, ReversibleLM(vocab_sz, 512),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy, perplexity, bpc]).to_native_fp16()
total_params(learn.model)

(19298051, True)

In [ ]:
learn.fit_one_cycle(1, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,1.747764,1.668458,0.495296,5.303983,2.407076,03:23


In [ ]:
learn.to_my_profile()
learn.fit_one_cycle(2, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,2.224247,2.110697,0.382028,8.253995,3.045093,03:24
1,1.914338,1.851212,0.447856,6.367532,2.670734,03:23


In [ ]:
learn.my_profile.print_stats()

fit  called 2 times. max: 407.500 avg: 305.371
   epoch  called 3 times. max: 204.154 avg: 203.576
      train  called 3 times. max: 180.776 avg: 180.162
         train_batch  called 1212 times. max: 0.465 avg: 0.341
            train_pred  called 1212 times. max: 0.182 avg: 0.131
            train_loss  called 1212 times. max: 0.004 avg: 0.001
            train_backward  called 1212 times. max: 0.236 avg: 0.179
            train_step  called 1212 times. max: 0.042 avg: 0.025
            train_zero_grad  called 1212 times. max: 0.014 avg: 0.005
      valid  called 3 times. max: 23.517 avg: 23.402
         valid_batch  called 285 times. max: 0.146 avg: 0.138
            valid_pred  called 285 times. max: 0.126 avg: 0.120
            valid_loss  called 285 times. max: 0.002 avg: 0.001


In [ ]:
learn.my_profile.clear_stats()

In [ ]:
del learn
torch.cuda.empty_cache()

### Using Irreversible blocks

In [ ]:
learn = Learner(dls, ReversibleLM(vocab_sz, 512, rev_thres=sl+1),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy, perplexity, bpc]).to_native_fp16()
total_params(learn.model)

(19298051, True)

In [ ]:
learn.fit_one_cycle(1, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,2.258606,2.211136,0.351905,9.126074,3.189994,03:53


In [ ]:
learn.to_my_profile()
learn.fit_one_cycle(2, 5e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time
0,1.927476,1.823154,0.455392,6.191355,2.630255,03:53
1,1.723771,1.657315,0.497695,5.245211,2.391001,03:52


In [ ]:
learn.my_profile.print_stats()

fit  called 1 times. max: 466.264 avg: 466.264
   epoch  called 2 times. max: 233.288 avg: 233.128
      train  called 2 times. max: 210.593 avg: 210.273
         train_batch  called 808 times. max: 0.437 avg: 0.420
            train_pred  called 808 times. max: 0.128 avg: 0.124
            train_loss  called 808 times. max: 0.002 avg: 0.001
            train_backward  called 808 times. max: 0.279 avg: 0.266
            train_step  called 808 times. max: 0.035 avg: 0.024
            train_zero_grad  called 808 times. max: 0.015 avg: 0.005
      valid  called 2 times. max: 23.005 avg: 22.846
         valid_batch  called 190 times. max: 0.143 avg: 0.138
            valid_pred  called 190 times. max: 0.126 avg: 0.120
            valid_loss  called 190 times. max: 0.002 avg: 0.001


In [ ]:
learn.my_profile.clear_stats()

In [ ]:
del learn
torch.cuda.empty_cache()